# Fake News Classifier Using Bidirectional LSTM

In [3]:
import pandas as pd 
df = pd.read_csv("fake_news.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# Check missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Drop the Nan values
df = df.dropna()

In [8]:
# Get the independent features and dependent feature
X = df.drop("label" , axis=1)
y = df.label

In [9]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
X.shape , y.shape

((18285, 4), (18285,))

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
# Vocabulary size
voc_size = 5000


## onehot Representation

In [14]:
messages = X.copy()

In [15]:
messages["title"][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [16]:
messages.reset_index(inplace=True)

In [17]:
# use some nlp related libraries to handle words

import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages["title"][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
    review = ' '.join(review)
    corpus.append(review)

1
14952
14953
14954
14955
14956
14957
14958
14959
14960
14961
14962
14963
14964
14965
14966
14967
14968
14969
14970
14971
14972
14973
14974
14975
14976
14977
14978
14979
14980
14981
14982
14983
14984
14985
14986
14987
14988
14989
14990
14991
14992
14993
14994
14995
14996
14997
14998
14999
15000
15001
15002
15003
15004
15005
15006
15007
15008
15009
15010
15011
15012
15013
15014
15015
15016
15017
15018
15019
15020
15021
15022
15023
15024
15025
15026
15027
15028
15029
15030
15031
15032
15033
15034
15035
15036
15037
15038
15039
15040
15041
15042
15043
15044
15045
15046
15047
15048
15049
15050
15051
15052
15053
15054
15055
15056
15057
15058
15059
15060
15061
15062
15063
15064
15065
15066
15067
15068
15069
15070
15071
15072
15073
15074
15075
15076
15077
15078
15079
15080
15081
15082
15083
15084
15085
15086
15087
15088
15089
15090
15091
15092
15093
15094
15095
15096
15097
15098
15099
15100
15101
15102
15103
15104
15105
15106
15107
15108
15109
15110
15111
15112
15113
15114
15115
15116
15117
15

In [20]:
corpus

chut new york time',
 'stranahan steve bannon nail media fight trump breitbart',
 'texa elector expect massiv corrupt relat elector colleg vote',
 'guilti power nullif counteract govern tyranni',
 'trillion new debt day',
 'tori councillor say homeless peopl elimin',
 'review hillbilli elegi tough love analysi poor back trump new york time',
 'crimin chief',
 'govern lie movi',
 'trump brexit defeat global anyway',
 'face congress sport offici begin confront sexual abus new york time',
 'airbnb end fight new york citi fine new york time',
 'big pharma martin shkreli suspend twitter breitbart',
 'next big tech corridor seattl vancouv planner hope new york time',
 'evid robot win race american job new york time',
 'see africa road new york time',
 'happen hip hop hillari goe dead broke brace',
 'bill maher high trump state free speech new era new york time',
 'l p g tour donald trump complic new york time',
 'legend art cashin trump presid new world order gold brexit great depress see pa

In [21]:
onehot_rep = [one_hot(words , voc_size) for words in corpus]
onehot_rep

97, 4541, 4281, 4865, 680, 3451],
 [2046, 1292, 1393, 517, 4194, 1787, 262, 4487, 87, 1887, 758, 4481],
 [124, 3968, 2983, 289],
 [3809, 3527, 1075, 943, 2682, 4328, 616, 4984],
 [1611, 3731, 3750, 1315, 1754, 4152, 1310, 1787, 1839, 4358],
 [3402, 408, 1477, 3516, 502, 811, 342, 1542],
 [3697, 4342, 3697, 2277, 1919, 4314],
 [459, 4659, 3920, 555, 3074, 511, 793, 4782, 2680, 1787, 1839, 4358],
 [2899, 286, 3339, 2293, 3635, 3740, 1746, 1578, 3389, 1787, 1839, 4358],
 [1491, 3068, 1047, 1890, 4315, 631, 3472, 3876],
 [2443, 3094, 3339, 3092, 4198, 2866, 4595, 802, 1787, 1839, 4358],
 [490, 1712, 1252, 3276, 3395],
 [2075, 3217, 3297, 3027, 4437, 310, 755, 428, 1911, 1787, 1839, 4358],
 [798, 124, 2433, 449, 3604, 3606, 1791, 3584, 2267],
 [244, 1580, 376, 61, 4734, 3362, 124],
 [4760, 517, 730, 4157, 124, 31, 1479, 3389],
 [3951, 3074, 1866, 2429, 1845, 3951],
 [124, 2562, 3876, 1977],
 [1279, 1312, 4913, 1791, 1663, 3037, 3302, 297, 1542, 3951],
 [638, 2646, 31, 3972, 3491, 4445, 124,

## Embedding Representation

In [22]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep , padding="pre" , maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  174 3791 2860]
 [   0    0    0 ... 2156 3186 3951]
 [   0    0    0 ...  883 1301 3034]
 ...
 [   0    0    0 ... 1787 1839 4358]
 [   0    0    0 ... 1338  888 2166]
 [   0    0    0 ... 3447 4915 1799]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1908,
       3537, 1746, 4073,  184,  543, 3871,  174, 3791, 2860])

## Creating a model

In [37]:
embedding_vector_features=40

model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
embedded_docs.shape ,y.shape

((18285, 20), (18285,))

In [39]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [40]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Training a model

In [42]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 86ms/step - loss: 0.2836 - accuracy: 0.8681 - val_loss: 0.2078 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 12s 64ms/step - loss: 0.1359 - accuracy: 0.9490 - val_loss: 0.2112 - val_accuracy: 0.9155
Epoch 3/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0953 - accuracy: 0.9634 - val_loss: 0.2875 - val_accuracy: 0.9157
Epoch 4/10
192/192 [==============================] - 13s 65ms/step - loss: 0.0692 - accuracy: 0.9748 - val_loss: 0.2802 - val_accuracy: 0.9137
Epoch 5/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0504 - accuracy: 0.9835 - val_loss: 0.3105 - val_accuracy: 0.9090
Epoch 6/10
192/192 [==============================] - 14s 73ms/step - loss: 0.0316 - accuracy: 0.9908 - val_loss: 0.3805 - val_accuracy: 0.9079
Epoch 7/10
192/192 [==============================] - 14s 73ms/step - loss: 0.0249 - accuracy: 0.9919 - val_loss: 0.4305 - val_accuracy:

## Performance Metrics And Accuracy

In [43]:
y_pred = model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [44]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test , y_pred)

array([[3117,  302],
       [ 282, 2334]], dtype=int64)

In [45]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test , y_pred)

0.9032311516155758

Accuracy is not bad and i think model perform better performance

In [47]:
from sklearn.metrics import classification_report

print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

